In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("DecisionTreeClassifier").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/22 10:30:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/22 10:30:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
df = spark.read.csv('avocado.csv',inferSchema=True,header='true')

In [7]:
colu = df.columns
print(colu)

['_c0', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region']


In [8]:
colu.index('region')

13

In [9]:
df.summary().toPandas()

22/08/22 10:31:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , AveragePrice, Total Volume, 4046, 4225, 4770, Total Bags, Small Bags, Large Bags, XLarge Bags, type, year, region
 Schema: _c0, AveragePrice, Total Volume, 4046, 4225, 4770, Total Bags, Small Bags, Large Bags, XLarge Bags, type, year, region
Expected: _c0 but found: 
CSV file: file:///Users/wirarama/python/FGA/BPDFGA/avocado.csv


,summary,_c0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,count,18249,18249,18249,18249,18249,18249,18249,18249,18249,18249,18249,18249,18249
1,mean,24.232231903117977,1.4059784097758825,850644.0130089332,293008.42453066056,295154.56835607596,22839.735992657315,239639.20205983953,182194.68669571026,54338.08814455636,3106.4265072058793,None,2016.1478985149872,None
2,stddev,15.48104475375712,0.4026765554955525,3453545.3553994684,1264989.0817627835,1204120.4011350533,107464.06843537069,986242.3992164108,746178.5149617895,243965.96454740898,17692.89465191642,None,0.9399384671420276,None
3,min,0,0.44,84.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,conventional,2015,Albany
4,25%,10,1.1,10830.75,853.38,3006.89,0.0,5078.45,2844.43,127.16,0.0,None,2015,None
5,50%,24,1.37,107306.08,8640.78,29054.71,184.62,39737.03,26339.6,2645.4,0.0,None,2016,None
6,75%,38,1.66,432903.0,110954.74,149994.79,6232.61,110773.22,83325.05,21945.2,132.23,None,2017,None
7,max,52,3.25,6.250564652E7,2.274361617E7,2.047057261E7,2546439.11,1.937313437E7,1.33845868E7,5719096.61,551693.65,organic,2018,WestTexNewMexico


In [11]:
df.select('region').collect()

[Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region='Albany'),
 Row(region

In [10]:
df.createOrReplaceTempView('avocado')

In [13]:
from pyspark.sql import functions as F

In [ ]:
"UPDATE nyc SET `LAND SQUARE FEET`=0 WHERE `LAND SQUARE FEET` like'%-%'"

In [14]:
def hapusStrip(x):
    return df.withColumn(x,F.when(F.col(x).like('%-%'),0).otherwise(F.col(x)))

In [15]:
df = hapusStrip('LAND SQUARE FEET')
df = hapusStrip('GROSS SQUARE FEET')

In [21]:
data=df.rdd.map(lambda x:(Vectors.dense(x[colu.index('Small Bags')],
                                        x[colu.index('Large Bags')],
                                        x[colu.index('XLarge Bags')]))
                .toDF(["region", "label"]))

In [22]:
data.show()

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [18]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel",handleInvalid='skip').fit(data)

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [20]:
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

AttributeError: 'PipelinedRDD' object has no attribute '_jdf'

In [21]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [22]:
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [23]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [24]:
model = pipeline.fit(trainingData)

In [25]:
predictions = model.transform(testData)

In [26]:
predictions.select("prediction", "indexedLabel", "features").show(5)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|[7.0,1002.0,0.0,0.0]|
|       1.0|         1.0|[15.0,1179.0,0.0,...|
|       1.0|         1.0|[15.0,1231.0,0.0,...|
|       1.0|         1.0|[15.0,1285.0,0.0,...|
|       1.0|         1.0|[15.0,1310.0,0.0,...|
+----------+------------+--------------------+
only showing top 5 rows



In [27]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.157029 


In [ ]:
df.select(['BOROUGH']).collect()

In [ ]:
treeModel = model.stages[2]

In [ ]:
print(treeModel)